# 数据清洗

In [ ]:
import pandas as pd
pd.set_option('display.float_format', '{:.2f}'.format)
# 设置每列最大宽度（比如 100）
pd.set_option('display.max_colwidth', 100)

# 设置显示的最大列数（比如 20）
pd.set_option('display.max_columns', 20)

# 设置显示宽度（比如 1000）
pd.set_option('display.width', 1000)

download = pd.read_csv('pd_data/user_download.csv', index_col=0)
all_info = pd.read_csv('pd_data/user_all_info.csv')

数据清洗（Data Cleaning）是数据预处理中的关键步骤，指的是识别并修复或删除数据中的错误、缺失、不一致或重复项，以确保数据的质量和可靠性。

## 什么是数据清洗

数据清洗是指对原始数据进行以下操作：
- 去除重复记录：如同一用户被记录多次。
- 填补缺失值：如某些字段为空或缺失。
- 纠正格式错误：如日期格式不一致、大小写混乱。
- 统一编码或单位：如“男/女”与“Male/Female”统一成一种格式。
- 识别异常值：如年龄为 999，收入为负数。
- 删除无效或脏数据：如乱码、非法字符、无意义字段。


## 为什么要进行数据清洗

数据清洗的目的在于提升数据质量，从而确保分析结果的准确性和模型的有效性。具体原因包括：
1. 提高数据分析的准确性
- 脏数据会导致统计结果偏差、误导决策。
2. 提升机器学习模型性能
- 模型训练依赖干净、结构化的数据，错误数据会降低预测效果。
3. 避免程序报错或崩溃
- 缺失值、格式错误可能导致代码运行失败。
4. 增强数据一致性与可读性
- 清洗后的数据更易于理解、展示和共享。
5. 满足业务或合规要求
- 某些行业（如金融、医疗）对数据质量有严格要求。


# 检测与处理重复值

数据重复是数据分析经常面对的问题之一。对重复数据进行处理之前，需要分析重复数据产生的原因以及去除这部分数据后可能造成的不良影响。

常见的重复数据分为两种：
- 记录重复：即一个或多个特征的某几条记录值完全相同
- 特征重复：即存在一个或多个特征名称不同，但数据完全相同

## 记录重复值

在user_download表中记录了用户对app的下载意愿

数据表格式如下：
 |用户编号|是否愿意下载|
 |-|-|
 |0|Yes|

### 利用list去重

In [ ]:
#使用列表去重
#去重函数
def del_rep(list1):
    list2 = []
    for i in list1:
        if i not in list2:
            list2.append(i)
    return list2

In [ ]:
download

In [ ]:
download_list = list(download['是否愿意下载']) 

In [ ]:
len(download_list)

In [ ]:
download_rep = del_rep(download_list)

In [ ]:
download_rep

### 利用set的特性去重

利用集合中元素的不能重复的特性进行去重

In [ ]:
download_set = set(download_list)

In [ ]:
download_set

### 利用list与set去重的比较

比较list与set方法可以发现，使用list方法显得代码冗长，会影响数据分析的整体进度。使用set去重代码简介了很多，但使用set去重会导致数据的排列发生改变

list方法与set方法去重后排列顺序的对比：
|源数据|list方法|set方法|
|-|-|-|
|Y|Y|Y
|N|||
|||N|

### 使用drop_duplicates()方法去重

由于list和set去重方法有一定缺陷，pandas提供了一个名为drop_duplicates的去重方法，使用该方法去重不会改变数据的原始排布，并且兼具代码简洁和运行稳定的特点
```
DataFrame.drop_duplicates(
    subset=None, 
    keep='first', 
    inplace=False, 
    ignore_index=False
)
```
- subset：指定列名或列名列表。默认 None，即检查所有列。
- keep：控制保留哪一个重复值。
    - 'first'：保留第一次出现的重复项（默认）。
    - 'last'：保留最后一次出现的重复项。
    - False：删除所有重复项。
- inplace：布尔值，是否在原 DataFrame 上直接修改。默认 False。
- ignore_index：布尔值，是否重置索引。默认 False。


In [ ]:
download_select = download['是否愿意下载'].drop_duplicates() 

In [ ]:
download_select

In [ ]:
all_info

In [ ]:
shape_det = all_info.drop_duplicates(subset = ['用户编号', '编号'])

In [ ]:
shape_det

In [ ]:
print(all_info.shape)
print(shape_det.shape)

## 特征重复

### 使用特征重复值去重

要去除连续的特征重复值，可以利用特征间的相似度从两个相似度为1的特征中去除一个

什么是相似度矩阵
- 定义：相似度矩阵（Similarity Matrix）是一个二维矩阵，用来表示一组对象之间的相似程度。
- 形式：通常是一个 对称矩阵，行和列代表同一组对象，矩阵中的元素表示两个对象之间的相似度。
- 取值范围：常见的相似度值在 0 ~ 1 之间，1 表示完全相同，0 表示完全不同。


在pandas中，相似度的计算方法为corr()。使用该方法计算相似度默认使用pearson方法，可以通过method参数进行调节，目前还支持spearman方法和kendall方法。

什么是 corr()
- 作用：计算 DataFrame 或 Series 的相关系数，衡量不同列之间的线性相关性。
- 返回值：一个相关系数矩阵（通常是对称矩阵）。
- 取值范围：相关系数一般在 -1 ~ 1 之间。
- 1 表示完全正相关
- -1 表示完全负相关
- 0 表示无线性相关

```
DataFrame.corr(method='pearson', min_periods=1, numeric_only=False)
```
参数说明
- method：相关系数的计算方法
    - 'pearson'（默认）：皮尔逊相关系数，衡量线性关系
    - 'kendall'：肯德尔相关系数，衡量排序一致性
    - 'spearman'：斯皮尔曼相关系数，基于秩次的相关性
- min_periods：计算相关系数所需的最少非缺失值数量
- numeric_only：是否只计算数值型列（默认 False，Pandas 会自动选择合适的列）


In [ ]:
# 求取年龄和每月支出的相似度
corr_det = all_info[['年龄', '每月支出']].corr(method='kendall')
# corr_det = all_info[['用户编号', '编号']].corr(method='kendall')

In [ ]:
corr_det

通过相似度矩阵去重存在的弊端之一是只能对数值型重复特征去重，类别型特征之间无法通过计算相似系数来衡量相似度，因此无法依据相似度矩阵去重。

In [ ]:
corr_det1 = all_info[['居住类型', '年龄', '每月支出']].corr(method='pearson')
#若不填写numeric_only参数会报错，原因是居住类型不为数值型数据类型

### 使用equals方法去重

方法作用
- 用于判断两个 DataFrame 或 Series 是否完全相同。
- 返回值是一个布尔值：
    - True → 两者在 形状、元素值、缺失值位置 都完全一致
    - False → 任意一点不同


```
DataFrame.equals(other)
Series.equals(other)
```
- other：另一个 DataFrame 或 Series，用来比较。


In [ ]:
# 定义求取特征是否完全相同的矩阵的函数
def feature_equals(df):
    df_equals = pd.DataFrame([])
    for i in df.columns:
        for j in df.columns:
            df_equals.loc[i, j] = df.loc[:, i].equals(df.loc[:, j])
    return df_equals

In [ ]:
app_desire = feature_equals(all_info)

In [ ]:
app_desire

In [ ]:
# 遍历所有数据
len_feature = app_desire.shape[0]   #输出app_desire的行数
dup_col = []    #- 初始化一个空列表，用来存储检测到的重复列名
for m in range(len_feature):
    for n in range(m + 1, len_feature):
        if app_desire.iloc[m, n] & (app_desire.columns[n] not in dup_col):
            dup_col.append(app_desire.columns[n])

```
if app_desire.iloc[m, n] & (app_desire.columns[n] not in dup_col):
```
- app_desire.iloc[m, n]：取出第 m 行、第 n 列的元素。
- app_desire.columns[n] not in dup_col：检查第 n 列的列名是否还没有被记录。
- &：这里用的是 按位与运算符，但在布尔逻辑中也常被用作“与”。
    - 如果 app_desire.iloc[m, n] 是布尔值或 0/1 数值，这里就相当于逻辑判断。

In [ ]:
dup_col

In [ ]:
all_info_drop1 =  all_info.drop(dup_col, axis=1, inplace=False)

In [ ]:
all_info_drop1

# 检测与处理缺失值

有时数据中的某个或某些特征的值是不完整的，这些值称为缺失值。

## 检测缺失值

pandas提供了isnull()方法和notnull()方法来识别缺失值和非缺失值。

这两种方法返回值都是布尔类型

结合sum函数可以检测数据中缺失值的分布以及数据中一共含有多少缺失值

In [ ]:
all_info.isnull().sum()

In [ ]:
all_info.notnull().sum()

isnull方法和notnull方法的结果正好相反，因此使用任意一个都可以识别出是否为缺失值。

## 处理缺失值

在检测出缺失值之后可以通过删除法、替换法或插值法等方法对缺失值进行处理

### 删除法

删除法是指将还有缺失值的特征或记录删除。

删除法分为删除记录和删除特征两种，它是通过减少样本数量来换取信息完整度的一种方法。是一种较为简单的处理缺失值的方法


pandas中提供率简便的删除缺失值的方法dropna()
```
DataFrame.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False)
```
参数说明
- axis：指定删除方向
    - 0 或 'index' → 删除行（默认）
    - 1 或 'columns' → 删除列
- how：删除条件
    - 'any'（默认）：只要有一个 NaN 就删除
    - 'all'：整行或整列全是 NaN 才删除
- thresh：保留至少多少个非 NaN 值，否则删除
    - 例如 thresh=2 表示至少要有 2 个非缺失值才保留
- subset：指定检查的列或行范围
    - 只在某些列上检查缺失值
- inplace：是否在原 DataFrame 上修改（默认 False）


In [ ]:
all_info.shape

In [ ]:
all_info1 = all_info.dropna(axis=0, how='any')

In [ ]:
all_info1.shape

### 替换法

替换法是指用一个特定的值替换缺失值。特征可分为数值型特征和类别型特征。两者出现缺失值时的处理方法也不同。

当缺失值为数值型特征时，通常利用其均值、中位数或众数等描述其集中趋势的统计两来代替缺失值

当缺失值为类别型特征时，则通常使用众数来替换缺失值。

pandas库中提供了替换缺失值的fillna()方法。
```
DataFrame.fillna(value=None, method=None, axis=None, inplace=False, limit=None)
```
- value：用来替换缺失值的数值或字典。
    - 可以是一个常数（如 0），也可以是一个字典（按列指定填充值）。
- method：填充方法。
    - 'ffill' 或 'pad' → 前向填充（用前一个非 NaN 值填充）。
    - 'bfill' 或 'backfill' → 后向填充（用后一个非 NaN 值填充）。
- axis：指定填充方向（默认按列）。
- inplace：是否在原 DataFrame 上修改（默认 False）。
- limit：限制填充的次数（例如只填充连续缺失值中的前几个）。


In [ ]:
# 求每月支出平均值
mean_num = all_info['每月支出'].mean()

In [ ]:
all_info2 = all_info

In [ ]:
all_info2['每月支出'] = all_info2['每月支出'].fillna(mean_num)

In [ ]:
all_info2['每月支出'].isnull().sum()

## 插值法

删除法会引起数据结构变动，使样本减少；替换法会影响数据的标准差，导致信息量变得。

面对数据缺失问题时，除了删除法和替换法外，还有一种常用方法——插值法

常用的插值法有线性插值法、多项式插值和样条插值等。
- 线性插值法针对已知的值求出线性方程，通过求解线性方程得到缺失值。
- 多项式插值法利用已知的值拟合一个多项式，使得现有的数据满足这个多项式，在利用这个多项式插值。常见的多项式又有拉格朗日插值法和牛顿插值法等方法。
- 样条插值法是以可变样条来做出一条经过一系列点的光滑曲线的插值法。插值样条由一些多项式组成，每一个多项式都由相邻两个数据点决定，这样可以保证两个相邻多项式及其导数在连接处连续。

pandas提供了名为interpolate的插值方法。但是Scipy库的interpolate库更加全面

scipy.interpolate 模块提供了多种插值方法，用于在已知离散数据点之间估算未知点的值。它支持 一维插值、样条插值、多维插值和径向基函数插值，常用于数据平滑、缺失值填补和函数近似。


In [ ]:
import numpy as np
# 创建自变量x
x = np.array([1, 2, 3, 4, 5, 8, 9, 10]) 
# 创建因变量y1
y1 = np.array([2, 8, 18, 32, 50, 128, 162, 200])  
# 创建因变量y2
y2 = np.array([3, 5, 7, 9, 11, 17, 19, 21]) 

自变量x与 $y_1$ 的关系为 $y_1=2x^2$

自变量x与 $y_2$ 的关系为 $y_2=2x+1$

#### 线性插值

In [ ]:
from scipy.interpolate import interp1d

In [ ]:
# 线性插值拟合x、y1
linear_ins_value1 = interp1d(x, y1, kind='linear')  

In [ ]:
# 线性插值拟合x、y2
linear_ins_value2 = interp1d(x, y2, kind='linear')  

In [ ]:
print('当x为6、7时，使用线性插值y1为：', linear_ins_value1([6, 7]))
print('当x为6、7时，使用线性插值y2为：', linear_ins_value2([6, 7]))

#### 拉格朗日插值

In [ ]:
from scipy.interpolate import lagrange

In [ ]:
# 拉格朗日插值拟合x、y1
large_ins_value1 = lagrange(x, y1)  

In [ ]:
# 拉格朗日插值拟合x、y2
large_ins_value2 = lagrange(x, y2)  

In [ ]:
print('当x为6,7时，使用拉格朗日插值y1为：', large_ins_value1([6, 7]))
print('当x为6,7时，使用拉格朗日插值y2为：', large_ins_value2([6, 7]))

### 样条插值

In [ ]:
# 样条插值拟合x、y1
y1_new = np.linspace(x.min(), x.max(), 10)
# 编辑插值函数格式
f = interp1d(x, y1, kind='cubic')  
# 通过相应的插值函数求得新的函数点
spline_ins_value1 = f(y1_new) 

In [ ]:
# 样条插值拟合x、y2
y2_new = np.linspace(x.min(), x.max(), 10)
# 编辑插值函数格式
f = interp1d(x, y2, kind='cubic')  
# 通过相应的插值函数求得新的函数点
spline_ins_value2 = f(y2_new) 

In [ ]:
print('使用样条插值y1为：', spline_ins_value1)
print('使用样条插值y2为：', spline_ins_value2)

# 检测与处理异常值

异常值是指数据中个别数值明显偏离其余的数值，有时也称为离群点。

检测异常值就是检验数据中是否由输入错误或不合理的数据。

## 3σ准则

拉依达准则（Pauta Criterion，又称 3σ准则）是一种用于剔除实验数据中 粗大误差（异常值） 的统计方法。

它假设数据服从正态分布，如果某个测量值与平均值的偏差超过 3倍标准差，就判定为异常值并剔除。


In [ ]:
all_info_notnull = pd.read_csv('pd_data/all_info_notnull.csv')

In [ ]:
all_info_notnull

In [ ]:
# 定义3σ原则来识别异常值函数
def out_range(ser1):
    bool_ind = (ser1.mean() - 3 * ser1.std() > ser1) | \
                (ser1.mean() + 3 * ser1.std() < ser1)
    index = np.arange(ser1.shape[0])[bool_ind]
    outrange = ser1.iloc[index]
    return outrange

In [ ]:
outlier = out_range(all_info_notnull['年龄'])

In [ ]:
print('使用3σ原则判定异常值个数为:', outlier.shape[0])
print('异常值的最大值为：', outlier.max())
print('异常值的最小值为：', outlier.min())

## 箱线图分析

箱线图（Boxplot，又称箱形图、盒须图）是一种统计图表，用于展示数据的分布特征。

它能直观反映数据的 中位数、四分位数、极值和异常值，常用于比较不同数据集的差异和识别异常值

箱线图的构成要素
- 中位数 (Q2)：箱体中间的横线，表示数据的中间位置。
- 下四分位数 (Q1)：箱体下边界，表示数据的 25% 分位点。
- 上四分位数 (Q3)：箱体上边界，表示数据的 75% 分位点。
- 四分位距 (IQR = Q3 - Q1)：箱体的高度，反映数据的波动范围。
- 须 (Whiskers)：从箱体延伸出去的线，通常表示在 Q1 − 1.5IQR 和 Q3 + 1.5IQR 范围内的最小值和最大值。
- 异常值 (Outliers)：落在须之外的点，用圆点或星号标记。


箱线图的作用
- 识别异常值：超出箱须范围的点就是潜在异常值。
- 判断分布偏态：中位数是否居中、箱体和须是否对称，可以反映数据的偏态。
- 比较多组数据：多个箱线图并排，可以直观比较不同组的分布差异。


In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(10, 8), dpi=1080) 
p = plt.boxplot(list(all_info_notnull['年龄'].values))  # 画出箱线图
outlier1 = p['fliers'][0].get_ydata()  # fliers为异常值的标签
plt.savefig('pd_tmp/用户年龄异常数据识别.jpg')
plt.show()
print('年龄数据异常值个数为：', len(outlier1))
print('年龄数据异常值的最大值为：', max(outlier1))
print('年龄数据异常值的最小值为：', min(outlier1))
